In [41]:
import pandas as pd
import numpy as np
from collections import Counter

import pyspark
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

from riotwatcher import LolWatcher, ApiError
import psycopg2

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl

In [14]:
# !pip install pyspark

In [2]:
#!pip install psycopg2
conn = psycopg2.connect(
                 dbname = 'postgres', 
                 user = 'dbuser', 
                 password = 'dbpassword',
                 host = '35.193.195.58',
                 port = '5432', 
                 sslmode = 'require',
                 sslrootcert='../data/ssl/server-ca.pem',
                 sslcert ='../data/ssl/client-cert.pem' ,
                 sslkey = '../data/ssl/client-key.pem'
                )

In [4]:
with conn.cursor() as cur:
    conn.autocommit = True
    cur.execute("CREATE TABLE summ_champ ("
                   "recId integer CONSTRAINT firstkey PRIMARY KEY,"
                   "summId integer NOT NULL," 
                   "champId integer NOT NULL)")
#     cur.execute('DROP DATABASE league_data')


In [ ]:
# cursor.execute('INSERT INTO summ_champ (game_id, summ_id, champ_id)'
#                'VALUES (1,2,3)')
# connection.commit()

# I. Riot API

In [16]:
key = 'RGAPI-0c01e301-d943-4ca0-abe0-4249ae163df4'

## A. API Object Initialization

In [17]:
watcher = LolWatcher(key)
rgn = 'na1'
platf = 'americas'

In [29]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

emptyRDD = spark.sparkContext.emptyRDD()
schema = StructType([
  StructField('recId', StringType(), False),
  StructField('summId', StringType(), False),
  StructField('champId', IntegerType(), False)
  ])
df = spark.createDataFrame(emptyRDD,schema)
df.printSchema()

root
 |-- recId: string (nullable = false)
 |-- summId: string (nullable = false)
 |-- champId: integer (nullable = false)



In [44]:
me = watcher.summoner.by_name(rgn, 'broswitbros')
x,y,z = gen_matches(rgn, me['id'])
zip(x,y,z)

## B. Defining functions to make API calls

In [19]:
def gen_matches(rgn, summoner_id, platf = 'americas',queue = 420, count = 20):

    player = watcher.summoner.by_id(rgn, summoner_id)

    matches = watcher.match.matchlist_by_puuid(region = platf, puuid = player['puuid'], queue = queue, count = count)

    rec_ids, summ_ids, champions = [], [], []
    for match in matches:
        match_detail = watcher.match.by_id(platf, match)
        #print(match_detail['info']['participants'])
        for part in match_detail['info']['participants']:
            #print(part)
            if part['puuid'] == player['puuid']:
                rec_ids.append(match+'_'+str(part['teamId'])+'_'+part['teamPosition'])
                summ_ids.append(summoner_id)
                champions.append(part['championId'])
        #champions.append(match_detail['info']['participants']['championId'])

    return [[i,j,k] for i,j,k in zip(rec_ids, summ_ids, champions)]

In [20]:
def gen_champ_table():
    latest = watcher.data_dragon.versions_for_region(rgn)['n']['champion']
    # Lets get some champions static information
    champ_data = watcher.data_dragon.champions(latest, False, 'en_US')['data']
    champ_list = [[i['key'],i['name']] for i in list(champ_data.values())]
    return champ_list

### B1. Retrieving list of all ranked players Masters-Challenger

In [49]:
chall = watcher.league.challenger_by_queue(rgn, 'RANKED_SOLO_5x5')
gm = watcher.league.grandmaster_by_queue(rgn, 'RANKED_SOLO_5x5')
mast = watcher.league.masters_by_queue(rgn, 'RANKED_SOLO_5x5')

In [63]:
def update_summ_ids(tier):
    summ_library = [[s['summonerId'],s['summonerName']] for s in tier['entries']]
    return summ_library

In [96]:
def load_player_champs(tier):
    tier_matches = []
    i = 1
    for summoner in tier['entries']:
        s = summoner['summonerId']     
        tier_matches.extend(gen_matches(rgn, s, count = 50))
        print(i, summoner['summonerName'])
        i+=1
    return tier_matches

In [97]:
records  = load_player_champs(chall)

1 Satoshi
2 CYD NGU
3 Tempos Time
4 sadsdasdasf
5 Outsmarted
6 Sheiden2
7 lamour de ma vié
8 EvanRL
9 Lyteskin
10 wong diff
11 Avano
12 Diomarr
13 Nzm
14 WayneDwops
15 Saico
16 Sum
17 Le Kachu
18 concept x
19 izuyui
20 AllGirlsRTheSame
21 KoKoNwoo
22 metalhydra273
23             Tçº 
24 Logia
25 Qitong
26 Kang Sae Byeok1
27 Airflash
28 SkyTec
29 rovex
30 DIG Spawn
31 Ramòn
32 Sophist Sage
33 ibarakura
34 Bejjj
35 loljanan
36 Wxx REB20201
37 TempestLoveYuely
38 wx mjm978244659
39 Oronuke
40 MONSTER CHAD GOD
41 Juheon
42 Blacc Massu
43 eg jojo
44 Lemur
45 bard kitten
46 Hakuho
47 Anticípation
48 Kennywalol1
49 Fake smiIes
50 Secret PIayer
51 VX Najuehuo
52 Breezyyy
53 Ariendel
54 liantiguaiwu
55 Doublelift
56 TailsJJ
57 Jouzef
58 TheRealPhilip
59 winstxn
60 DarbBarf
61 Dragoon
62 saving na
63 Mei You Qing Xu
64 wx NAtongtiandai
65 Eric Wei
66 Dark Wingdom
67 Emprisonner
68 Delicate Angel
69 THE Jons
70 SpazzAD
71 waste it on me
72 blonde dahyun
73 Trevor26
74 DeadGemini
75 Pobelter
76 jo

In [94]:
champ_table = gen_champ_table()

[['266', 'Aatrox'],
 ['103', 'Ahri'],
 ['84', 'Akali'],
 ['166', 'Akshan'],
 ['12', 'Alistar'],
 ['32', 'Amumu'],
 ['34', 'Anivia'],
 ['1', 'Annie'],
 ['523', 'Aphelios'],
 ['22', 'Ashe'],
 ['136', 'Aurelion Sol'],
 ['268', 'Azir'],
 ['432', 'Bard'],
 ['53', 'Blitzcrank'],
 ['63', 'Brand'],
 ['201', 'Braum'],
 ['51', 'Caitlyn'],
 ['164', 'Camille'],
 ['69', 'Cassiopeia'],
 ['31', "Cho'Gath"],
 ['42', 'Corki'],
 ['122', 'Darius'],
 ['131', 'Diana'],
 ['119', 'Draven'],
 ['36', 'Dr. Mundo'],
 ['245', 'Ekko'],
 ['60', 'Elise'],
 ['28', 'Evelynn'],
 ['81', 'Ezreal'],
 ['9', 'Fiddlesticks'],
 ['114', 'Fiora'],
 ['105', 'Fizz'],
 ['3', 'Galio'],
 ['41', 'Gangplank'],
 ['86', 'Garen'],
 ['150', 'Gnar'],
 ['79', 'Gragas'],
 ['104', 'Graves'],
 ['887', 'Gwen'],
 ['120', 'Hecarim'],
 ['74', 'Heimerdinger'],
 ['420', 'Illaoi'],
 ['39', 'Irelia'],
 ['427', 'Ivern'],
 ['40', 'Janna'],
 ['59', 'Jarvan IV'],
 ['24', 'Jax'],
 ['126', 'Jayce'],
 ['202', 'Jhin'],
 ['222', 'Jinx'],
 ['145', "Kai'Sa"],
 [

In [111]:
# spark.createDataFrame(records, schema).show()

# II. Building Recommender System

## A. Vectorizing Dataset

In [42]:
from sklearn.feature_extraction.text import CountVectorizer
from scipy.linalg import svd

In [43]:
cv = CountVectorizer()

In [75]:
# x_train = my_dict.values()
# x_train = [' '.join(row) for row in x_train]

# df = pd.DataFrame(cv.fit_transform(x_train).toarray(), columns = cv.get_feature_names()
df_vec = pd.crosstab(df.summId, df.champId)

In [ ]:
#df.to_csv('vect_champ_data.csv')
# import json

# with open('chall_dict.json','w') as fp:
#     json.dump(my_dict,fp)

## B. Applying SVD

### B1. Initialization

In [76]:
def svd_trans(df):
    df_vec = pd.crosstab(df.summId, df.champId)
    U, Sigma, VT = svd(df_vec)
    VT_ = VT.copy()
    VT = VT[:3,:]
    VT = pd.DataFrame(VT)
    return U, Sigma, VT, VT_

### B2. Visualizing similarity in 3-dimensional space

In [63]:
U, Sigma, VT, VT_ = svd_trans(df_vec)

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
plt.style.use('seaborn')

fig = plt.figure(figsize=(20,16))
ax = fig.gca(projection='3d')
ax.scatter(U[:,0],U[:,1],U[:,2],c='k',s=150);
ax.set_xlabel("D1", fontsize=20, labelpad=20)
ax.set_ylabel("D2", fontsize=20, labelpad=20)
ax.set_zlabel("D3", fontsize=20, labelpad=20);

lbls = df.index
offset = 0.01
for i, txt in enumerate(lbls):
    if i not in [6,7]:
        ax.text(U[i,0]+offset,U[i,1],U[i,2],txt, fontsize=20)
    else:
        ax.text(U[i,0]+offset,U[i,1],U[i,2]+5*offset,txt, fontsize=20)

In [ ]:
fig = plt.figure(figsize=(20,16))
ax = fig.gca(projection='3d')
ax.scatter(VT_.T[:,0],VT_.T[:,1],VT_.T[:,2],c='b',s=150, label="Items");
ax.scatter(U[:,0],U[:,1],U[:,2],c='r',s=150, label="Users");
ax.set_xlabel("D1", fontsize=20, labelpad=20)
ax.set_ylabel("D2", fontsize=20, labelpad=20)
ax.set_zlabel("D3", fontsize=20, labelpad=20);

lbls = df.columns
item_offset = 0.01
for i, txt in enumerate(lbls):
    if i not in [6,7]:
        ax.text(VT_.T[i,0],VT_.T[i,1]+item_offset,VT_.T[i,2],txt, fontsize=20)
    else:
        ax.text(VT_.T[i,0],VT_.T[i,1]+item_offset,VT_.T[i,2]+5*item_offset,txt, fontsize=20)

lbls = df.index
offset = 0.01
for i, txt in enumerate(lbls):
    if i not in [6,7]:
        ax.text(U[i,0],U[i,1]+offset,U[i,2],txt, fontsize=20)
    else:
        ax.text(U[i,0],U[i,1]+offset,U[i,2]+6*offset,txt, fontsize=20)
ax.view_init(30,15)
plt.legend(loc="upper left", fontsize=30);

## C. Evaluating similarity

In [89]:
def compare_champs(compare_champ, df, VT):
    sim_champ = {}
    for champ in cv.get_feature_names():
        if champ != compare_champ:
            t1 = VT.T[VT.T.index.str.startswith(compare_champ)].to_numpy()
            t2 = VT.T[VT.T.index.str.startswith(champ)].to_numpy()
            sim_champ["Champion %s & %s: "%(compare_champ,champ)] = np.dot(t1[0],t2[0])
    return sim_champ

In [138]:
def compare_users(compare_user, df, U):
    df_vec = pd.crosstab(df.summId, df.champId)
    lst_users = df_vec.index
    sim_player = {}
    for user in lst_users:
        if user != compare_user:
            user_idx = df_vec.index.get_loc(user)
            compare_idx = df_vec.index.get_loc(compare_user)
            sim_player["User %s & %s: "%(compare_user,user)] = np.dot(U[compare_idx],U[user_idx])
    return sim_player

In [ ]:
#print(max(sim_player, key = sim_player.get),max(sim_player.values()))

In [ ]:
#print(max(sim_champ, key = sim_champ.get),max(sim_champ.values()))

In [47]:
df = pd.read_csv('../data/final_data.csv').iloc[:,2:]

In [137]:
# pd.DataFrame(U,index = df_vec.index)
df_vec.index

Index(['-HeyTwZqvfwQrwAfR7pXZCIZu2g5uAnP_xdishsH0waKF-A',
       '-ZToIdKib8TpMCZs6pAawH0970zFy4v9sadIl3qyYuEE6Cc',
       '-d5YahFMxlTqI1A8VjiOrSgnj02Nbry5eajClqYcSU6Pn80X',
       '-etPMsImiyI_skY7taENVBW2EmrfcmeFIqVWyOzuVGojo1RJl6RW2tr21Q',
       '-fZYORfIVBUYHtLHdMW1xn_hQ8IBbu8ydpI9k94WyDKAdpc',
       '-kGZpOoEA430aLjkol0SJfYkg-OJgiEmjftLs1Z8kCcCj4I',
       '-mVjyEQHT9YBIPiJYi0Y1TpannMC0nAEGQp7lJNkrqLAKzM',
       '-xeMtvJpZ4Fd8XXBDefnh4BHGVVNXoabLn4G4FSkQgImAOU',
       '-zUS-SQEhvenl4ra2O0RVdZW36A238eIx5x-Ahz18mD3qEg',
       '00bixxc18hAEeo_eEiCzoqYhL3Grd4-7j2DhctHQUP03x5k',
       ...
       'xP7IT9O2eAtg6jiadFTDb-VHCb5t2KMC1UL2NumlgSVo1zSNhDEKAWiu2g',
       'xp5K6NuFYORW8osCGrIa71l1nDEMznQpP31dgJcuGkxeE0w',
       'yAzduuTP7cI0vXhw-gLX-7K_eAwGaB_fWnaFawRJay7LclPB',
       'yC_Lyz6COoSCmobhOG0ZsoOO6GntSZilQ_7qGsKvitr25Rw',
       'yNztZqKbeiO3JpsY6clCqpGxuCdjDQ72-UNNsi2f8B2RXdU',
       'ye0KtxysiDSMMVWPUZ8ewuHqbpMPKOeSLnH12dWNKczTCIE',
       'yoQfDLXBv9eb7s-Mi5x1KQ5Jnk1SG

# III. Recommending

## A. Baseline recommender using KNN

In [21]:
me = watcher.summoner.by_name(rgn, 'broswitbros')
my_pool = gen_matches(rgn, me['id'], queue = None, count = 20)

[['NA1_4142438955_200_UTILITY',
  'ZoQWkA_DQZqEBHudNeqAp7tlWpB-pE0b3MMZr8hNOr3mlYc',
  111],
 ['NA1_4142481448_200_JUNGLE',
  'ZoQWkA_DQZqEBHudNeqAp7tlWpB-pE0b3MMZr8hNOr3mlYc',
  163],
 ['NA1_4142385261_200_JUNGLE',
  'ZoQWkA_DQZqEBHudNeqAp7tlWpB-pE0b3MMZr8hNOr3mlYc',
  163],
 ['NA1_4141500783_200_TOP',
  'ZoQWkA_DQZqEBHudNeqAp7tlWpB-pE0b3MMZr8hNOr3mlYc',
  6],
 ['NA1_4141416889_200_BOTTOM',
  'ZoQWkA_DQZqEBHudNeqAp7tlWpB-pE0b3MMZr8hNOr3mlYc',
  202],
 ['NA1_4139814306_100_BOTTOM',
  'ZoQWkA_DQZqEBHudNeqAp7tlWpB-pE0b3MMZr8hNOr3mlYc',
  202],
 ['NA1_4138978542_200_MIDDLE',
  'ZoQWkA_DQZqEBHudNeqAp7tlWpB-pE0b3MMZr8hNOr3mlYc',
  38],
 ['NA1_4139022202_200_MIDDLE',
  'ZoQWkA_DQZqEBHudNeqAp7tlWpB-pE0b3MMZr8hNOr3mlYc',
  38],
 ['NA1_4138879578_100_MIDDLE',
  'ZoQWkA_DQZqEBHudNeqAp7tlWpB-pE0b3MMZr8hNOr3mlYc',
  38],
 ['NA1_4138874811_100_JUNGLE',
  'ZoQWkA_DQZqEBHudNeqAp7tlWpB-pE0b3MMZr8hNOr3mlYc',
  104],
 ['NA1_4138860365_200_MIDDLE',
  'ZoQWkA_DQZqEBHudNeqAp7tlWpB-pE0b3MMZr8hNOr3mlYc',
  1

In [141]:
my_df = pd.DataFrame(columns = ['gameId','summId','champId'])
for row in my_pool:
    my_df = my_df.append(pd.Series(row, index=my_df.columns), ignore_index = True)

In [142]:
my_df = df.append(my_df).iloc[:,:2]
U, sigma, VT,_ = svd_trans(my_df)

In [144]:
sim_player = compare_users(me['id'], my_df, U)
# sim_champ = compare_champs('kassadin', my_df, VT)

print(max(sim_player, key = sim_player.get),max(sim_player.values()))
# print(max(sim_champ, key = sim_champ.get),max(sim_champ.values()))


User ZoQWkA_DQZqEBHudNeqAp7tlWpB-pE0b3MMZr8hNOr3mlYc & Lc-Z7Wq3plXoiceJ8G9qn4YYME0k9FR7VF3NCPLQDZ8wJ3o:  3.469446951953614e-16


In [ ]:
Counter(gen_champ_dict(gen_champ_ids(rgn, me['id'], queue = None)))

In [ ]:
print(chall['entries'][153]['summonerName'], ': ', Counter(my_dict[chall['entries'][153]['summonerName']]))

## B. NCF - Neural Collaborative Filtering

In [ ]:
df = pd.read_csv('vect_champ_data.csv')

In [ ]:
users = []
champs = []
for i in range(len(df)):
    summ_row = df.iloc[i][1:]
    count_lst = [(summ_row.index[idx],item) for idx,item in enumerate(summ_row) if item != 0]
    nested_lists = [[champ]*count for champ, count in count_lst]
    flat = [item for sublist in nested_lists for item in sublist]
    users.extend([i]*len(flat))
    champs.extend(flat)

In [ ]:
df_trans = pd.DataFrame()
df_trans['summoner'], df_trans['champ'] = users, champs
all_champs = df_trans['champ'].unique()
df_trans['champ_id'] = df_trans['champ'].map(lambda x: np.where(all_champs == x)[0][0])

In [ ]:
champ_ids = np.array(range(len(all_champs)))
num_negatives = 4
summ_champ_set = set(zip(df_trans['summoner'], df_trans['champ_id']))
s, c, l = [],[],[]

for (u, i) in summ_champ_set:
    s.append(u)
    c.append(i)
    l.append(1)
    for _ in range(num_negatives):
        neg_champ = np.random.choice(champ_ids)
        while (u, neg_champ) in summ_champ_set:
            neg_champ = np.random.choice(champ_ids)
        s.append(u)
        c.append(neg_champ)
        l.append(0)

    

In [ ]:
df_adjust = pd.DataFrame()
df_adjust['summoners'], df_adjust['champion'], df_adjust['labels'] = s,c,l
x,y,z = torch.tensor(s),torch.tensor(c),torch.tensor(l)

In [ ]:
model = pl.LightningModule()

In [ ]:
num_summs = len(set(s))
num_champs = len(set(c))

model.summoner_embedding = nn.Embedding(num_embeddings = num_summs, embedding_dim=8)
model.champion_embedding = nn.Embedding(num_embeddings = num_champs, embedding_dim=8)
model.fc1 = nn.Linear(in_features=16, out_features=64)
model.fc2 = nn.Linear(in_features=64, out_features=32)
model.output = nn.Linear(in_features=32, out_features=1)

In [ ]:
trainer = pl.Trainer(max_epochs=5, gpus=0, reload_dataloaders_every_epoch=True,
                     progress_bar_refresh_rate=50, logger=False, checkpoint_callback=False)

In [ ]:
torch.cuda.is_available()
torch.__version__

In [ ]:
!pip install torch==1.10.1+cu113 torchvision==0.11.2+cu113 torchaudio===0.10.1+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html